<a href="https://colab.research.google.com/github/ee0122ng/EBAC_Semester2_AirBnbDataAnalysis/blob/main/LSTM_BinaryClassifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import datasets
import pandas as pd

df_hm6A = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hm6A_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hm6Am = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hm6Am_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hPsi = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hPsi_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hm7G = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hm7G_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hm5U = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hm5U_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hTm = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hTm_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hm5C = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hm5C_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_Atol = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/Atol_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hGm = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hGm_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hAm = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hAm_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hm1A = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hm1A_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')
# df_hCm = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/hCm_with_ROS.csv', usecols=lambda x: x!='Unnamed: 0')

In [ ]:
df_hm6A.shape

(309214, 102)

### Word2vec Model Training

In [ ]:
# define function to extract kmer from the original dataset
from itertools import product

nucleotide = ['A', 'C', 'G', 'T', 'N']

def get_ngrams(seq, k):
  for i in range(len(seq) - k + 1):
    yield seq[i:i+k]

def extract_ngrams(data, k):
  combinations = sorted([''.join(str(s) for s in t) for t in product(nucleotide, repeat=k)])
  ngrams = []
  ngrams_map = {}

  # get list of nucleotide combinations and populate to count
  for g in get_ngrams(data, k):
    ngrams.append(g)
    if g in ngrams_map:
      ngrams_map[g] += 1
    else:
      ngrams_map[g] = 1

  return combinations, ngrams, ngrams_map

# extract k-mer
def extract_kmer(k, data):

  # extract 2grams from the processed dataset
  ngrams = [*map(extract_ngrams, data.sum(axis=1).tolist(), [k]*data.shape[0])]

  # transform 2grams features into dataframe
  features = [*map(lambda x: x[1], ngrams)]
  column = [*map(lambda x: 'p_'+str(x), range(len(features[0])))]
  df = pd.DataFrame(features, columns=column)

  return df


In [ ]:
import pandas as pd

# import original dataset
df_raw = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data/train_in.csv', usecols=lambda x : x != 'Unnamed: 0')
df = df_raw.iloc[:, 450:550]

# extract k-mer
df_kmer = extract_kmer(2, df)

# hold values in a list as corpus
corpus_RNA = df_kmer.values.tolist()

KeyboardInterrupt: ignored

In [ ]:
df_kmer.to_csv('/content/drive/My Drive/EBAC_Capstone/data/train_in_2mer_withNoise.csv', index=False)

In [ ]:
# train Word2Vec model with the corpus
import gensim
from gensim.models import Word2Vec

# train the model
# vector_size = number of features to produce by the NN
# window = window for skip-grams (forward and backward from the target word)
# min_count = minimum number of word used in the model
# worker = number of parallel works
%time model = Word2Vec(sentences=corpus_RNA, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# save the Word2Vec model with noise
model.save('/content/drive/My Drive/EBAC_Capstone/model/word2vec_model_withNoise.bin')

In [ ]:
# load trained word2vec model
import gensim

word2vec = gensim.models.word2vec.Word2Vec.load("/content/drive/My Drive/EBAC_Capstone/model/word2vec_model_withNoise.bin")

In [ ]:
# get 2mer nucleotide combinations
from itertools import product

nucleotide = ['A', 'C', 'G', 'T', 'N']
combinations = sorted([''.join(str(s) for s in t) for t in product(nucleotide, repeat=2)])

In [ ]:
for c in combinations:
  try:
    if word2vec.wv[c].shape is not None:
      print('%s: value' %c)
    else:
      print('%s: null' %c)
    print('-'*30)
  except:
    print('%s: null' %c)
    print('-'*30)

### binary classification for multi classes

In [ ]:
# train LSTM model from features embedded via Word2Vec on 2mer dataset
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from itertools import product
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.checkpoint import checkpoint
import os
from torch.cuda.amp import GradScaler, autocast # Automatic Mixed Precision (AMP)

# enable GPU
gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define a class to build LSTM model
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
    super(LSTMModel, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)
    self.activation = nn.ReLU()

  def forward(self, x):
    x = x.float()
    x = x.to(gpu)
    # h0 and c0 are used to initialize the lstm model
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(gpu)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(gpu)
    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])
    # out = self.fc(out[:, -1])
    return out

# define a class to build trainloader
class MyDataset(Dataset):
  def __init__(self, features, targets):
    self.features = features
    self.targets = targets

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.targets[idx]

'''
sub-function to extract k-mer dataset from the original dataset
'''
nucleotide = ['A', 'C', 'G', 'T', 'N']

def get_ngrams(seq, k):
  for i in range(len(seq) - k + 1):
    yield seq[i:i+k]

def extract_ngrams(data, k):
  combinations = sorted([''.join(str(s) for s in t) for t in product(nucleotide, repeat=k)])
  ngrams = []
  ngrams_map = {}

  # get list of nucleotide combinations and populate to count
  for g in get_ngrams(data, k):
    ngrams.append(g)
    if g in ngrams_map:
      ngrams_map[g] += 1
    else:
      ngrams_map[g] = 1

  return combinations, ngrams, ngrams_map

'''
sub-function for train_test_split
'''
def convert_to_word2vec_features(data, k):
  print('embedding features...')
  nucleotide = ['A', 'C', 'G', 'T', 'N']

  # load word2vec model
  model = Word2Vec.load('/content/drive/My Drive/EBAC_Capstone/model/word2vec_model_withNoise.bin')

  # encode the 2mer dataset with new features generated through gensim.models.word2vec
  mapping_word2vec = {}
  combinations = sorted([''.join(str(s) for s in t) for t in product(nucleotide, repeat=k)])
  for c in combinations:
    # combination might not seen by word2vec embedding
    try:
      mapping_word2vec[c] = model.wv[c]
    except:
      continue

  cols = data.columns
  df_wv_encoded = pd.DataFrame()
  for c in cols:
    df_wv_encoded[c] = data[c].map(mapping_word2vec)

  return df_wv_encoded

def convert_dataframe_to_tensors(X_train, X_valid, y_train, y_valid):
  print('coverting to tensors...')
  # convert dataframe to numpy array
  arr_train = np.array(X_train.values.tolist(), dtype=np.float16)
  arr_valid = np.array(X_valid.values.tolist(), dtype=np.float16)
  arr_train_y = np.array(y_train.ravel().tolist(), dtype=np.int16)
  arr_valid_y = np.array(y_valid.ravel().tolist(), dtype=np.int16)

  # convert the datasets to Pytorch tensors
  X_train = torch.tensor(arr_train, dtype=torch.float16)
  X_valid = torch.tensor(arr_valid, dtype=torch.float16)
  y_train = torch.tensor(arr_train_y, dtype=torch.int16)
  y_valid = torch.tensor(arr_valid_y, dtype=torch.int16)

  # # convert the datasets to Pytorch tensors
  # concatenated_Xtrain = None
  # concatenated_Xvalid = None
  # concatenated_ytrain = None
  # concatenated_yvalid = None

  # batchSize = 250
  # for i in range(0, arr_train.shape[0], batchSize):
  #   batch_Xtrain = arr_train[i: i+batchSize]
  #   batch_Xvalid = arr_valid[i: i+batchSize]
  #   batch_ytrain = arr_train_y[i: i+batchSize]
  #   batch_yvalid = arr_valid_y[i: i+batchSize]

  #   if concatenated_Xtrain is None:
  #     concatenated_Xtrain = torch.tensor(batch_Xtrain, dtype=torch.float32)
  #     concatenated_Xvalid = torch.tensor(batch_Xvalid, dtype=torch.float32)
  #     concatenated_ytrain = torch.tensor(batch_ytrain, dtype=torch.float32)
  #     concatenated_yvalid = torch.tensor(batch_yvalid, dtype=torch.float32)
  #   else:
  #     concatenated_Xtrain = torch.cat((concatenated_Xtrain, torch.tensor(batch_Xtrain, dtype=torch.float32)), dim=0)
  #     concatenated_Xvalid = torch.cat((concatenated_Xvalid, torch.tensor(batch_Xvalid, dtype=torch.float32)), dim=0)
  #     concatenated_ytrain = torch.cat((concatenated_ytrain, torch.tensor(batch_ytrain, dtype=torch.float32)), dim=0)
  #     concatenated_yvalid = torch.cat((concatenated_yvalid, torch.tensor(batch_yvalid, dtype=torch.float32)), dim=0)

  return X_train, X_valid, y_train, y_valid
  # return concatenated_Xtrain, concatenated_Xvalid, concatenated_ytrain, concatenated_yvalid

def split_dataset(data):
  print('splitting dataset...')
  # transform features with word2vec
  # get word2vec embedding for k=2
  df_X = convert_to_word2vec_features(data.iloc[:, :-1], 2) # have dependent column insert to the last
  df_y = data.iloc[:, -1]

  # split the dataset into train and validation sets
  X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(df_X, df_y, random_state=123, test_size=0.3)

  # convert dataframe to tensors
  X_train, X_valid, y_train, y_valid = convert_dataframe_to_tensors(X_train, X_valid, y_train, y_valid)

  return X_train, X_valid, y_train, y_valid

def prepare_dataloaders(X_train, X_valid, y_train, y_valid, batchSize=64):
  print('preparing dataloader...')
  # get trainloader
  train_dataset = MyDataset(X_train, y_train)
  trainloader = DataLoader(train_dataset, shuffle=True, num_workers=4, batch_size=batchSize)

  # get validloader
  valid_dataset = MyDataset(X_valid, y_valid)
  validloader = DataLoader(valid_dataset, shuffle=False, num_workers=4, batch_size=batchSize)

  return trainloader, validloader

def train_LSTM_model(df, hidden_size, num_layers, output_size, num_epochs, batch_size, datasetName):

  os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

  # split dataframe into featrues and output
  features = df.iloc[:, :-1]
  outputs = df.iloc[:, -1].to_frame()

  # check if 2mer dataset is ready
  df = pd.DataFrame()
  try:
    df = pd.read_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/encoded/' + 'train_in_' + datasetName + '.csv', usecols=lambda x: x!='Unnamed: 0')
    print('dataframe %s available' %datasetName)
  except:
    # transform features to 2mer
    result_2g = [*map(extract_ngrams, features.sum(axis=1).tolist(), [2]*features.shape[0])]
    features_2g = [*map(lambda x: x[1], result_2g)]
    df_2mer = pd.DataFrame(features_2g)

    # convert outputs to machine readable
    outputs.loc[outputs['TARGET'] == 'NonMoD', 'TARGET'] = 0
    outputs.loc[outputs['TARGET'] != 0, 'TARGET'] = 1

    # reform the dataset
    df = pd.concat([df_2mer, outputs], axis=1)

    # export dataset
    df.to_csv('/content/drive/My Drive/EBAC_Capstone/data_binary/encoded/' + 'train_in_' + datasetName + '.csv', index=False)

  # enable GPU
  gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # free up memory
  torch.cuda.empty_cache()

  # step1: prepare and split the data
  X_train, X_valid, y_train, y_valid = split_dataset(df)

  # step2: prepare dataloader
  trainloader, validloader = prepare_dataloaders(X_train, X_valid, y_train, y_valid, batchSize=batch_size)

  # create an instance of LSTM model
  # input_size = number of input neurons
  # hidden_size = number of hidden neurons
  # num_layer = number of LSTM layer
  # output_size = number of output neurons
  input_size = X_train[0].shape[1]
  print('training model...')
  model = LSTMModel(input_size, hidden_size, num_layers, output_size)

  # define loss function and optimizer
  # criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)

  # train a LSTM model
  length = len(trainloader)

  scaler = GradScaler()

  for epoch in range(num_epochs):
    model.train() # set model to training mode
    total_loss = 0

    for i, data in enumerate(trainloader, 0):
      inputs, targets = data
      optimizer.zero_grad()

      # move input, target and model to gpu
      model = model.to(gpu)
      inputs = inputs.to(gpu)
      targets = targets.to(gpu)

      # # Reshape the target tensor to [batch_size, 1]
      targets = targets.float()
      targets = targets.view(-1, 1)

      with autocast():
        outputs = model(inputs)
        loss = criterion(outputs, targets)

      scaler.scale(loss).backward()
      scaler.step(optimizer)
      scaler.update()
      total_loss += loss.item()

      # free up resources
      torch.cuda.empty_cache()

    # calculate average loss for the epoch
    avg_loss = total_loss / (len(trainloader) // batch_size)

    # evaluate on the validation set
    model.eval() # set model to evaluation mode
    with torch.no_grad():
      model = model.to(gpu)
      X_valid = X_valid.to(gpu)

      y_valid = y_valid.to(gpu)
      y_valid = y_valid.float()
      y_valid = y_valid.view(-1, 1)

      val_outputs = model(X_valid)
      val_outputs = torch.relu(val_outputs) # use the activation function to convert the outputs
      val_loss = criterion(val_outputs, y_valid)

      # free up resources
      torch.cuda.empty_cache()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}, Val Loss: {val_loss.item()}")
  return model

In [ ]:
# evaluate LSTM model
import torch
import torch.nn as nn
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix as cfm
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(model, X_test, y_test):

  criterion = nn.BCEWithLogitsLoss()

  # evaluate on the validation set
  model.eval()
  with torch.no_grad():
    model = model.to(gpu)
    X_test = X_test.to(gpu)

    y_test = y_test.to(gpu)
    y_test = y_test.float()
    y_test = y_test.view(-1, 1) # convert shape from [1] to [1,1]

    val_outputs = model(X_test)

  # Move tensors to CPU and convert to NumPy arrays
  val_outputs = val_outputs.cpu()
  y_test = y_test.cpu()

  # evaluate the accuracy
  # predicted = torch.argmax(val_outputs, dim=1)
  predicted_relu = torch.relu(val_outputs)
  predicted = (predicted_relu > 0.5).float()
  accuracy = (predicted == y_test).float().mean()
  print('Accuracy: {:.2f}%'.format(accuracy * 100))

  # precision tp / (tp + fp)
  precision = precision_score(y_test, predicted)
  print('Precision: %f' % precision)

  # recall: tp / (tp + fn)
  recall = recall_score(y_test, predicted)
  print('Recall: %f' % recall)

  # f1: 2 tp / (2 tp + fp + fn)
  f1 = f1_score(y_test, predicted)
  print('F1 score: %f' % f1)

  # visualize confusion matrix
  cnf_matrix = cfm(y_test, predicted)
  class_names=[0,1] # name  of classes
  fig, ax = plt.subplots()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names)
  plt.yticks(tick_marks, class_names)

  # create heatmap
  sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
  ax.xaxis.set_label_position("top")
  plt.tight_layout()
  plt.title('Confusion matrix', y=1.1)
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

### train model on each subset

In [ ]:
'''
********** hm6A **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hm6A, hidden_size=16, num_layers=2, output_size=1, num_epochs=20, batch_size=64, datasetName='hm6A')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hm6A.pth')

# evaluate performance for hm6A
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hm6A)

# get accuracy
evaluate_model(model, X_valid, y_valid)

dataframe hm6A available
splitting dataset...
embedding features...
coverting to tensors...
preparing dataloader...
training model...


In [ ]:
!nvidia-smi

Wed Jul 12 14:43:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |  13655MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
'''
********** hm6Am **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hm6Am, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hm6Am')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hm6Am.pth')

# evaluate performance for hm6Am
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hm6Am)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hPsi **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hPsi, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hPsi')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hPsi.pth')

# evaluate performance for hPsi
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hPsi)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hm7G **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hm7G, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hm7G')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hm7G.pth')

# evaluate performance for hm7G
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hm7G)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hm5U **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hm5U, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hm5U')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hm5U.pth')

# evaluate performance for hm5U
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hm5U)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hTm **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hTm, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hTm')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hTm.pth')

# evaluate performance for hTm
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hTm)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hm5C **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hm5C, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hm5C')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hm5C.pth')

# evaluate performance for hm5C
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hm5C)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** Atol **********
'''
# train LSTM model
%time model = train_LSTM_model(df_Atol, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='Atol')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_Atol.pth')

# evaluate performance for Atol
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_Atol)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hGm **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hGm, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hGm')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hGm.pth')

# evaluate performance for hGm
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hGm)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hAm **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hAm, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hAm')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hAm.pth')

# evaluate performance for hAm
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hAm)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hm1A **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hm1A, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hm1A')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hm1A.pth')

# evaluate performance for hm1A
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hm1A)

# get accuracy
evaluate_model(model, X_valid, y_valid)

In [ ]:
'''
********** hCm **********
'''
# train LSTM model
%time model = train_LSTM_model(df_hCm, hidden_size=100, num_layers=2, output_size=1, num_epochs=20, batch_size=150, datasetName='hCm')

# save model
torch.save(model, '/content/drive/My Drive/EBAC_Capstone/model/03 LSTM_epoch20_hidden100_layer2_batch150/LSTM_hCm.pth')

# evaluate performance for hCm
# get valid dataset
X_train, X_valid, y_train, y_valid = split_dataset(df_hCm)

# get accuracy
evaluate_model(model, X_valid, y_valid)